In [ ]:
import pypdf
import chromadb
import urllib3
import accelerate
import sentence_transformers
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
%env TOKENIZERS_PARALLELISM=True
%env TF_CPP_MIN_LOG_LEVEL=3

## Extracting text data from PDF Files

In [ ]:
# Create an object to load PDF file
loader = PyPDFLoader('../data/ArtigoDSA1.pdf')

type(loader)

In [ ]:
# Load the file PDF
pages = loader.load()

pages

In [ ]:
page = pages[2]

print("Page content: ", page.page_content[0:500])

In [ ]:
print("Metadata:", page.metadata)

## Splitting Text Data in Chunks

**chunk_size = 1000**: Specifies that each resulting chunk of text will have a maximum of 1000 characters.

**chunk_overlap = 20**: Indicates that each chunk will have 20 characters of overlap with the next chunk. This means that the last 20 characters of a chunk will be repeated at the beginning of the next chunk.

What it is for:

This approach is useful in several situations where large texts need to be processed or analyzed, such as:

- Input for language models: Many LLMs have a limit of tokens that they can process in a single iteration. Dividing the text into smaller chunks ensures that the text is sent within the allowed limit.

- Data analysis and indexing: It is common in search engines and data processing pipelines, where dividing the text into smaller chunks makes it easier to index and retrieve information.

- Context maintenance: When processing involves long documents, this technique allows you to deal with them more efficiently, by dividing the parts without losing logic or cohesion.

In [7]:
# Create the chunk text separator
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 20)

In [ ]:
# Applying the object and extracting the chunks (documents)
docs = splitter.split_documents(pages)

print("Total of Chunks (Documents):", len(docs))

print("Last Chunk Content (Document):", docs[6])

---

## Loading Text Data Vectors into the Vector Database

The code implements a semantic search system using a vector database (vectordb) to identify the most relevant points in relation to a question, based on the semantic similarity between the documents and the provided question.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

https://www.trychroma.com/

In [11]:
# Create the vector database
vectordb = Chroma.from_documents(documents = docs,
                                 embedding = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2"),
                                 persist_directory = "vectordb/chroma/")

**Chroma.from_documents(documents=docs)**: Creates a vector database using the provided documents (stored in the docs variable). These documents can be texts, articles, or any type of textual data that you want to index.

**HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")**: Uses the Hugging Face all-MiniLM-L6-v2 semantic embedding model to transform texts into numeric vectors. Embeddings are mathematical representations of texts that capture their semantic meaning.

**persist_directory="dsavectordb/chroma/"**: Specifies the directory where the vector database will be saved (persisted), so that it can be reused in future sessions without having to reprocess the documents.

In [ ]:
# Total collections in vector db
vectordb._collection.count()